# Using Genetic Algorithms to Form Optimal Groups

We have all been in that situation. You have to do a group project in a class, and the lecturer assigns the groups randomly. You find yourself in a group with people you don’t know and end up doing all the work by yourself. We want to change that and want you to implement an Genetic Algorithm that optimizes groups.


## Task 1:


You have to implement the initial population, crossover, mutation, and fitness function. We divided it into smaller subtasks for you and implemented some of the functions for you. If you want to do your own implementation, feel free to ignore our hints and functions.

Your initial population consists of 50 group distributions, where 100 students are assigned to 20 groups. Each student has a Student ID, a name, their spoken language, their 2 Majors (we assume everyone is doing their Masters), their ambition in the course, their preferred meeting place, their personality type, their gender, a friend that they want to be in a group with, and their preferred meeting day. 

To get you even more involved with the task, we want every member of your group to add their own person into the data. To do so, you simply have to execute the Jupyter Notebook named "Dataset_Input.ipynb". We are using the Myers-Briggs personality, which may not be the most scientific, but it for sure is entertaining. If you don’t know your type, you can take the test here (approx. 10 minutes) https://www.16personalities.com/free-personality-test. 

Run this cell to load the packages:

In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib as plt
import ipywidgets
import IPython
from Generator import DataGenerator
from typing import List

Execute this cell to create the dataset:

In [3]:
generator = DataGenerator()

#df, type_ = generator.create_basic()

df, type_ = generator.create_full()

df.to_csv("dataset_" + type_ + ".csv", encoding="utf-8-sig", index=False)

Hyperparameters:

In [ ]:
students: pd.DataFrame = pd.read_csv("dataset_full.csv")
student_ids: List[int] = students.ID.tolist()

# hyperparameters
num_individuals: int = 50
groupsize: int = 5
# between 0 and 1
mutation_rate: float = 0.05

students.head()

<class 'pandas.core.frame.DataFrame'>


,ID,Name,Gender,Preferred language,Majors,Level of ambition,Preferred meeting place,Personality type,Best friend,Preferred day
0,1,Jennifer Kortig,Female,Any,"('AI', 'NI')",High,Online,INFP,Kathrin Cole,Friday
1,2,Patrick Eichel,Male,Any,"('AI', 'NI')",Low,In person,INTJ,Melanie Bach,Monday
2,3,Ralph Schmid,Male,Any,"('AI', 'CL')",Medium,In person,ENTP,Ursula Hirsch,Friday
3,4,Christina Schreiber,Female,English,"('NI', 'CL')",Low,In person,INFJ,Markus Neumann,Tuesday
4,5,Johanna Cole,Female,Any,"('CP', 'AI')",Very low,In person,ESTP,Johanna Fenstermacher,Wednesday


We create one random individual:

In [ ]:
def create_random_individual(student_ids: List[int]) -> List[int]:
    #You don't need to do anything here.

    individual = student_ids.copy()
    random.shuffle(individual)

    return individual

print(create_random_individual(student_ids))

[57, 22, 100, 73, 45, 49, 77, 74, 83, 23, 85, 46, 47, 19, 3, 81, 98, 55, 95, 26, 31, 68, 27, 99, 66, 40, 62, 92, 93, 38, 9, 6, 8, 33, 84, 43, 72, 18, 89, 11, 87, 79, 14, 56, 97, 90, 91, 10, 21, 51, 13, 36, 7, 29, 20, 17, 70, 44, 60, 67, 88, 54, 78, 5, 48, 94, 69, 30, 2, 32, 58, 86, 4, 25, 12, 61, 64, 52, 82, 59, 75, 37, 63, 42, 28, 16, 80, 1, 53, 35, 50, 24, 39, 96, 15, 76, 65, 41, 34, 71]




Create the initial population of 50 (=num_individuals) individuals:


In [ ]:
def create_initial_population(students: pd.DataFrame, groupsize: int) -> np.ndarray:
    
    #should return a numpy array of the whole population
    pass

We need a fitness function that computes how good the group distribution is. You have to take into consideration that we have many parameters that are not equally important and should therefore be differently weighted. We already coded some of the specific evaluation functions for the parameters for you. You have to implement the remaining evaluation functions. Think about what is desirable to have in a group and how you can calculate it. For example, you might have to change the data type of a parameter to make meaningful calculatons.

At the end, you have to weigh all evaluation functions in one fitness function. 

In [ ]:
def evaluate_language(group: pd.DataFrame) -> float:
    pass

In [ ]:

def evaluate_majors(group: pd.DataFrame) -> float:
    #Remember that the majors are stored as a string like "('NS','AI')". 
    # You might need to preprocess those to actual tupels.
    pass


In [ ]:

def evaluate_ambition(group: pd.DataFrame) -> float:
    #it might be useful to convert the strings to an integer scala
    
    pass


In [ ]:

def evaluate_meeting_place(group: pd.DataFrame) -> float:
    #You don't need to code anything in here.
    #This is an example evaluation function.
    
    # number of groupmembers for each preferred meeting place
    meeting_place = group['Preferred meeting place'].value_counts()

    # if all prefer the same meeting place return 5, else 0
    if meeting_place[0] == groupsize:
        return 5

    return 0

In [ ]:
def evaluate_gender(group: pd.DataFrame) -> float:
    pass

In [ ]:
def evaluate_friends(array: pd.DataFrame, students: pd.DataFrame) -> float:
    pass

In [ ]:
def evaluate_personality(group: pd.DataFrame) -> float:
    #You dont need to change anything in here 

    #information about compatible personality types is taken from
    # Montequín, Vicente Rodríguez, et al. "Using Myers-Briggs type indicator (MBTI) as a tool for setting up student teams for information technology projects." Journal of Information Technology and Application in Education 1.1 (2012): 28-34.

    #count existing personality types in each group
    personalities = group['Personality type']
    types = personalities.value_counts()

    #fitness function starts with 0 and gets better
    # with every good group member
    fitness = 0

    #its good if there is a group leader like an ISTJ or an ESTJ, but only one
    try:
        if (types['ISTJ'] + types['ESTJ'] == 1):
            fitness+=5
        elif (types['ISTJ'] + types['ESTJ'] >= 2):
            fitness-=5
    except KeyError:
        pass

    #compare compatibility of group members
    for i, personality_a in enumerate(personalities.tolist()):
        for j, personality_b in enumerate(personalities.tolist()):
            # skip same group member and members already compared
            if i <= j:
                continue

            # increase fitness if
            if (personality_a[1] != personality_b[1]) ^ (personality_a[2] != personality_b[2]):
                if (personality_a[0] != personality_b[0]) or (personality_a[3] != personality_b[3]):
                    fitness+=1

    return fitness

In [ ]:
def evaluate_meeting_day(group: pd.DataFrame) -> float:
    pass

And now put everything together.
The function is almost done, but remember to add the weights.

In case you want to add hard constraints, feel free to do that in this function.



In [ ]:
def evaluate_fitness(individual: np.ndarray, students: pd.DataFrame):
    # split individual into student groups of the groupsize
    groups = np.array_split(individual, (len(individual)/groupsize))

    # iterate over groups and calculate scores for the different parameters
    scores = []
    for group_ids in groups:
        # get full data for students in this group from pd dataframe
        group = students.loc[students['ID'].isin(group_ids)]

        # get individual scores for parameters
        language_score = evaluate_language(group)
        major_score = evaluate_majors(group)
        ambition_score = evaluate_ambition(group)
        place_score = evaluate_meeting_place(group)
        gender_score = evaluate_gender(group)
        friend_score = evaluate_friends(group)
        personality_score = evaluate_personality(group)
        day_score = evaluate_meeting_day(group)

        # formula for adding and weighting different scores
        scores.append(language_score+major_score+ambition_score+place_score+gender_score+friend_score+personality_score+day_score)

    #Convert to series to calculate mean more easily
    return pd.Series(scores).mean()
    

Now, you have to code a crossover function, which takes 2 individuals based on their fitness function and produces a child from them. Use the tournament selection for parent selection.
For the crossover, we want you to use the uniform crossover function with random templates. 

In [ ]:
def tournament_selection(population: np.ndarray, tournament_size: int) -> List[np.ndarray]:
    

    #should return 2 parents as a list
    pass


Use a boolean template in the length of the individual, this can be hardcoded or generated randomly every time to add more variance. On the places where the template is true use the genes from parent1, then take all the genes from parent2 that are not used and add them to the empty places in the child in the same order as they appear on parent2

In [ ]:
def uniform_order_crossover(parent1: np.ndarray, parent2: np.ndarray, template: np.ndarray) -> np.ndarray:
    pass

Almost done! Now you have to implement the mutation function. It should take the individual produced by the crossover function and mutate it with a chance of for example 5%. A mutation switches the assigned groups of 2 people. 

In [ ]:
def mutation(individual: np.ndarray, mutation_rate: float) -> np.ndarray:
    pass


Now you have to write the overall genetic algorithm function, which uses all the functions you already implemented: 

In [4]:
#episodes is the number of episodes after the algorithm stops
#num_replace is the number of unfit individuals that will be replaced
def genetic_algorithm(episodes: int, num_replace: int) -> np.ndarray:
    
    
    
    
    pass

In [ ]:
population = genetic_algorithm(20,2)

### Let's find your group or the one of a friend

In [5]:
def get_groups_from_individual(individual: np.ndarray) -> List[np.ndarray]:
    """ Returns a Python List of all Groups in an individual(as numpy ndarrays) """
    groups = []
    for i in individual:
        groups.append(students.loc[i-1])

    nested_groups_list = []
    for n,j in enumerate(range(0, len(groups), groupsize)):
        nested_groups_list.append(pd.DataFrame(groups[j:j+5]))

    return nested_groups_list

def get_groups_by_person_fullname(fullname: str, individual: np.ndarray) -> List[np.ndarray]:
    """ Returns all Groups that got a Person with $fullname$ in given $individual$"""
    nested_groups_list = get_groups_from_individual(individual)

    groups_with_person = []
    for group in nested_groups_list:
        if fullname in group["Name"].values:
            groups_with_person.append(group)
    return groups_with_person

def get_group_by_person_ID(ID: int, individual: np.ndarray) -> np.ndarray:
    """ Returns Group that has Person with given $ID$ in given $individual$"""
    nested_groups_list = get_groups_from_individual(individual)

    groups_with_person = []
    for group in nested_groups_list:
        if ID in group["ID"].values:
            return group
    return None

#### Find your group by your Fullname

In [6]:
######### ATTENTION ##########
# Define your individual here!!! At the moment it will take a random individual! Pick for example the fittest individual.
individual = create_random_individual(student_ids)

# User Input for the Full Name - Please be accurate!
fullname = str(input("Please the Full-Name you want to look up: "))
# Get the group by your Full Name - If there are multiple Persons with your name you will get all of them
my_group = get_groups_by_person_fullname(fullname, individual)
# Plotting the first group found
if my_group:
    print(my_group)
else:
    print("Didn't find the requested Person with given Fullname")

NameError: name 'create_random_individual' is not defined

#### Find your group by your ID

In [ ]:
# Find your group by your ID
my_group = get_group_by_person_ID(33, individual)
# Plotting the group, if one found with given ID
if my_group is not None:
    print(my_group)
else:
    print("Didn't find the requested Person with given ID")

    ID              Name         Gender Preferred language          Majors  \
86  87        Marko Wulf  Indeterminate                Any    ('NS', 'CL')   
92  93  Michael Reinhard           Male            English    ('CL', 'NS')   
78  79       Lucas Hertz           Male                Any  ('CL', 'PHIL')   
95  96     Annett Peters         Female            English    ('AI', 'CP')   
32  33   Thorsten Schmid           Male                Any    ('AI', 'CL')   

   Level of ambition Preferred meeting place Personality type  \
86         Very high                  Online             INTP   
92         Very high               In person             ISTP   
78            Medium               In person             ENTP   
95          Very low               In person             ENTP   
32         Very high               In person             ENTP   

        Best friend Preferred day  
86     Sven Hertzog        Monday  
92   Birgit Hofmann       Tuesday  
78  Benjamin Kruger     Wednesda

## Task 2

Play around with the different values like initial population size, mutation rate, fitness function, and number of students and observe when it works the best. Write your insights down here.